# Australia Maximum Temperature Data Viewer

This notebook loads and visualizes the mock meteorological data stored in zarr format.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

# Set plot style
plt.style.use('default')
%matplotlib inline

## Load the Dataset

In [ ]:
# Load the zarr dataset
ds = xr.open_zarr('../data/australia_tmax.zarr')
print(ds)

## Dataset Information

In [ ]:
# Display metadata
print("\nDataset Attributes:")
for key, value in ds.attrs.items():
    print(f"  {key}: {value}")

print("\nTemperature Variable Attributes:")
for key, value in ds['tmax'].attrs.items():
    print(f"  {key}: {value}")

## Visualize Temperature Map (By Day)

Select a day to visualize the temperature distribution across Australia.

In [ ]:
# Load Australia boundary
aus_boundary = gpd.read_file('../data/australia_boundary.geojson')

# Select which day to visualize (0-9)
selected_day = 6  # Change this to view different days (0 = first day, 9 = last day)

# Get temperature data for selected day
tmax_day = ds['tmax'].isel(time=selected_day)
date_str = str(ds.time.values[selected_day])[:10]

# Create a figure
fig, ax = plt.subplots(figsize=(14, 10))

# Plot the temperature data
im = ax.pcolormesh(
    ds.lon, 
    ds.lat, 
    tmax_day,
    cmap='RdYlBu_r',  # Red-Yellow-Blue reversed (hot to cold)
    shading='auto',
    vmin=15,
    vmax=45
)

# Plot Australia boundary on top
aus_boundary.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5, label='Australia')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.8)
cbar.set_label('Temperature (°C)', fontsize=12)

# Set labels and title
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)
ax.set_title(f'Australia - Daily Maximum Temperature\nDay {selected_day}: {date_str}', 
             fontsize=14, fontweight='bold')

# Set aspect ratio to make it look more geographic
ax.set_aspect('equal')

# Add grid
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

# Add legend
ax.legend(loc='upper right')

# Add text showing max temperature for this day
max_temp = float(tmax_day.max())
mean_temp = float(tmax_day.mean())
ax.text(0.02, 0.98, f'Max: {max_temp:.1f}°C\nMean: {mean_temp:.1f}°C', 
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## Access Specific Location

Example: Get temperature at a specific coordinate

In [ ]:
# Example coordinates (roughly Sydney area)
example_lat = -33.9
example_lon = 151.2

# Get temperature time series at this location across all 10 days
point_temps = ds['tmax'].sel(lat=example_lat, lon=example_lon, method='nearest')

# Get actual coordinates
actual_lat = float(point_temps.lat)
actual_lon = float(point_temps.lon)

# Extract temperature values for each day
temps = [float(point_temps.isel(time=i)) for i in range(len(ds.time))]
dates = [str(t)[:10] for t in ds.time.values]
days = range(len(ds.time))

# Create time series plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(days, temps, 'o-', color='darkred', linewidth=2.5, markersize=10, markerfacecolor='red', markeredgecolor='darkred')

# Highlight the selected day
ax.plot(selected_day, temps[selected_day], 'o', markersize=15, markerfacecolor='yellow', 
        markeredgecolor='black', markeredgewidth=2, zorder=5, label=f'Selected Day ({selected_day})')

ax.set_xlabel('Day', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)
ax.set_title(f'10-Day Temperature at Location ({actual_lat:.2f}°, {actual_lon:.2f}°)\nNear Sydney', 
             fontsize=14, fontweight='bold')
ax.set_xticks(days)
ax.set_xticklabels([f'Day {i}\n{dates[i][5:]}' for i in days], fontsize=9)

# Set y-axis limits to accommodate labels
temp_range = max(temps) - min(temps)
ax.set_ylim(min(temps) - temp_range * 0.1, max(temps) + temp_range * 0.15)

ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

# Add value labels on each point
for i, temp in enumerate(temps):
    ax.text(i, temp + 0.3, f'{temp:.1f}°C', ha='center', fontsize=9, color='darkred')

plt.tight_layout()
plt.show()

print(f"\nLocation: ({actual_lat:.2f}°, {actual_lon:.2f}°)")
print(f"Temperature range: {min(temps):.1f}°C to {max(temps):.1f}°C")
print(f"Temperature change: {temps[-1] - temps[0]:.1f}°C (Day 0 to Day 9)")